In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import re

#Merge datasets

In [ ]:
movies = pd.read_csv('/content/gdrive/MyDrive/moviedata/movie/ir0.csv')

In [ ]:
movies = movies.replace('-', np.nan)
movies = movies.dropna()
movies = movies.dropna(subset=['year'])

In [ ]:
movies.info()

In [ ]:

for i in range(6, 135, 5):
  print(i)
  movie = pd.read_csv(f'/content/gdrive/MyDrive/moviedata/movie/ir{i}.csv')
  movies = pd.concat([movies, movie])
  movies = movies.replace('-', np.nan)
  movies = movies.dropna()
  movies = movies.dropna(subset=['year'])

In [ ]:
movies.head(10)

In [ ]:
movies.drop(['vote', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [ ]:
movies.drop_duplicates(inplace=True)

In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33069 entries, 0 to 4662
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   movie                  33069 non-null  object 
 1   year                   33069 non-null  object 
 2   runtime                32998 non-null  object 
 3   imdb_rating            33042 non-null  object 
 4   release_date           32999 non-null  object 
 5   plot_summary           33069 non-null  object 
 6   genres                 33068 non-null  object 
 7   budget                 15750 non-null  object 
 8   gross_earning          18965 non-null  object 
 9   opening_weekend_gross  11602 non-null  object 
 10  audience_score         25915 non-null  float64
 11  tomatometer_score      16494 non-null  float64
 12  rotten_info            31731 non-null  object 
 13  cast                   31696 non-null  object 
dtypes: float64(2), object(12)
memory usage: 3.8+ MB


In [ ]:
movies.to_csv('movies0.csv')

In [ ]:
movies.loc[movies['movie']=='From Justin to Kelly']

,movie,year,runtime,imdb_rating,release_date,plot_summary,genres,budget,gross_earning,opening_weekend_gross,audience_score,tomatometer_score,rotten_info,cast
2999,From Justin to Kelly,(2003),81 min,1.9,"June 20, 2003 (United States)",\nA waitress from Texas and a college student ...,"\nComedy, Musical, Romance","$12,000,000 (estimated)","$4,928,883","$2,715,848",39.0,8.0,\n\nRating:\nPG (Thematic Elements|Sensuality|...,\n\n\n\n\n\n\n\n\n\n Kelly ...


In [ ]:
!cp -r '/content/movies1.csv' '/content/gdrive/MyDrive/moviedata/movies1.csv'


# Clean dataset

In [1]:
movies = pd.read_csv('/content/gdrive/MyDrive/moviedata/movies0.csv')
usd = pd.read_csv('/content/gdrive/MyDrive/moviedata/usd.csv')
award = pd.read_csv('/content/gdrive/MyDrive/moviedata/oscar_award.csv')
picture = pd.read_csv('/content/gdrive/MyDrive/moviedata/picture_involve.csv')

NameError: ignored

In [ ]:
movies.drop(['Unnamed: 0'], axis = 1, inplace=True)
movies['year'] = movies['year'].str.extract('(\d+)').astype(int)
movies['runtime'] = movies['runtime'].str.extract('(\d+)').astype(int)
movies['gross_earning'] = movies['gross_earning'].map(lambda x: x.lstrip('$').replace(',',''))
movies['gross_earning'] = movies['gross_earning'].astype(int)
movies['opening_weekend_gross'] = movies['opening_weekend_gross'].map(lambda x: x.lstrip('$').replace(',',''))
movies['opening_weekend_gross'] = movies['opening_weekend_gross'].astype(int)
movies['budget'] = movies['budget'].str.replace('\D+', '').astype(int)
movies['genres'] = movies['genres'].str.replace('\n','')

In [ ]:
# clean 'rotten_info' column, extract director and mpaa info

for idx in movies.index:
  if 'Director' in movies.loc[idx,'rotten_info'].replace('  ',''):
    infos = movies.loc[idx,'rotten_info'].replace('  ','').split('\n\n\n\n')
    for x in infos:
      if 'Director' in x:
          movies.loc[idx,'rotten_info'] = x.split(':')[1].replace('\n','')
      if 'Rating' in x: 
        movies.loc[idx, 'mpaa'] = x.split(':')[1].split(' ')[0].replace('\n','')
  else:
    movies.loc[idx,'rotten_info'] = np.nan

In [ ]:
# clean 'cast' column: because structure of text not similar so i only get first actor
for idx in movies.index:
  a = movies['cast'].iloc[idx].replace('  ','').split('\n')
  b = []
  for i in a:
    if i != '':
      b.append(i)
  movies.loc[idx,'cast'] = b[0]

In [ ]:
# account budget, open weekend gross and box office gross for inflation
for idx in movies.index:
  idx_usd = usd.index[usd['year']==movies.loc[idx, 'year']].to_list()[0]
  movies.loc[idx, 'budget'] = movies.loc[idx,'budget'] * usd.loc[idx_usd, 'usd']
  movies.loc[idx, 'gross_earning'] = movies.loc[idx,'gross_earning'] * usd.loc[idx_usd, 'usd']
  movies.loc[idx, 'opening_weekend_gross'] = movies.loc[idx,'opening_weekend_gross'] * usd.loc[idx_usd, 'usd']

In [ ]:
picture.drop(['Unnamed: 0'], axis = 1, inplace=True)
picture['year'] = picture['year'].str.extract('(\d+)').astype(int)

In [ ]:
# clean the number of award that actor and director achieved and the number of best picture film they are involved 
for idx in movies.index:
  director = movies.loc[idx, 'rotten_info']
  actor = movies.loc[idx, 'cast']
  year = movies.loc[idx, 'year']
  sum = len(picture.loc[(picture['year'] <= year) & picture['name'].isin([director])])
  sum += len(picture.loc[(picture['year'] <= year) & picture['name'].isin([actor])])
  sum += len(award.loc[(award['year'] <= year) & award['name'].isin([director])])
  sum += len(award.loc[(award['year'] <= year) & award['name'].isin([actor])])
  movies.loc[idx, 'award_involve'] = sum

In [ ]:
movies.info()

In [ ]:
movies.loc[movies['mpaa'].isnull(), 'mpaa'] = 'NR'

In [ ]:
movies['award_involve'].unique()

In [ ]:
movies.to_csv('movies.csv', index=False)

In [ ]:
!cp -r '/content/movies.csv' '/content/gdrive/MyDrive/moviedata'